In [1]:
import os
os.chdir('../..')

In [2]:
import epicas

Hello there! Welcome back. 

Previously, we have loaded, merged, and performed some feature engineering techniques on our data. In this lab, let's try to get some predictions out of it! In this lab, we will use `epicas.Ensemble` to automatically initialize, train, and predict new cases using pre-specified list of models.

Let's begin!

## Unweighted Average Ensemble Modeling

Before we start, let's load data into StructuredData and EpiData (what we have done in the previous labs). If you have not read them, please do check them out here:
- [1. Loading and Merging](https://github.com/caominhduy/epicas/docs/ipynb/1_loading_and_merging.ipynb)
- [2. Feature Engineering](https://github.com/caominhduy/epicas/docs/ipynb/2_feature_engineering.ipynb)

The next code block will be hidden as it is identical to what we have already covered. Feel free to expand it!

In [3]:
jhu = epicas.StructuredData(
        'demo/datasets/covid.xz',
        location = 'FIPS',
        date = 'date',
        incidence = 'confirmed_cases',
        )

mobility = epicas.StructuredData(
        'demo/datasets/mobility.csv.gz',
        location = 'FIPS',
        date = 'date'
        )

population = epicas.StructuredData(
        'Reichlab_Population.csv',
        location = 'location',
        usecols = ['location', 'population']
        )

merged = jhu + mobility + population

merged = epicas.EpiData(merged, y='incidence', disease='covid19').imputation().target_to_ma(window=3)

merged = merged.normalization(subset=['population', 'fb_movement_change', 'fb_stationary'])

merged = merged.lag_reduction(subset=['fb_movement_change', 'fb_stationary'], sliding_window=21)


Optimal shift for fb_movement_change: 10
Optimal shift for fb_stationary: 10


This lab is a beautiful demonstration of AutoML: everything in this one should be automated!

**Disclaimer:** In this version of Epicas (0.1.0), we are using the most naive approach to ensemble modeling, which is Unweighted Average Modeling. All predictions from all models we specify will be averaged as our final predictions. We are expecting many more effective methods coming soon (including weighted average, random forest, and model-agnostic and game theoretic explainers like LIME and SHAP). If these sound cool, stay tune!

### Date Format

To specify the goal end date, you will need to use either `datetime` or a string of date in ISO-8601 format. 

For example, for September 1, 2021, you can specify as:

In [4]:
'2021-09-01'

'2021-09-01'

or

In [5]:
from datetime import date
date(2021,9,1)

datetime.date(2021, 9, 1)

### Models

At the moment, Epicas offers 5 different options:

- `attention`: Self-Attention Based Bidirectional LSTM models
- `ARIMA`: AutoRegressive Intergrated Moving Average
- `LSTM`: Long Short-term Memory
- `BiLSTM` or `Bi-LSTM`: Bidirectional LSTM (similar to LSTM, but going in forward and backward directions)
- `GRU`: Gated Recurrent Unit. A "simpler variant" of LSTM, released in 2014 by Kyunghyun Cho.

![GRU meme](https://i.imgflip.com/5powos.jpg)

For `Ensemble`, feel free to pick one, some, or all of these! To keep this lab neat, let's try only two models: Attention-based Bi-LSTM and ARIMA.

In [6]:
ensembled = epicas.Ensemble(merged, ['attention', 'ARIMA'], '2021-09-01')

Attention-based Bi-LSTM Fitting:   0%|                                                         | 0/410 [00:00<?, ?it/s]

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14, 4)]           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 14, 64)            9472      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 14, 64)            24832     
_________________________________________________________________
seq_self_attention (SeqSelfA (None, 14, 64)            4097      
_________________________________________________________________
flatten (Flatten)            (None, 896)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 3588      
Total params: 41,989
Trainable params: 41,989
Non-trainable params: 0
_________________________________________________________

ARIMA Fitting: 100%|███████████████████████████████████████████████████████████████| 2777/2777 [28:11<00:00,  1.64it/s]


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  424
Model:                ARIMA(12, 1, 0)   Log Likelihood               -1263.651
Date:                Fri, 08 Oct 2021   AIC                           2553.302
Time:                        04:39:40   BIC                           2605.918
Sample:                             0   HQIC                          2574.093
                                - 424                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.1031      0.037     30.196      0.000       1.031       1.175
ar.L2         -0.0202      0.066     -0.309      0.758      -0.149       0.108
ar.L3         -0.9259      0.064    -14.517      0.0

ARIMA Predicting: 100%|███████████████████████████████████████████████████████████| 2777/2777 [00:11<00:00, 235.30it/s]


Notice that if ARIMA seems being stuck at 0%, it is not. Before training the network, Epicas needs to run a few \[Augmented Dickey-Fuller\] and some step-wise fitting tests on the first batch to find the best hyperparameters (p,d) to feed into other batches. Be patient, the rest should be done pretty quickly...

**Note:** if you wonder why we have 410 batches to train for LSTM while ARIMA has 2777. Basically, for ARIMA, there is a different model for every locations, so we have 2777 high-quality locations to train on. Meanwhile, for LSTM, in each batch, the model runs on all locations, but we are looking at stepsize of 14 (14-day lookback for 1-day future predictions), so there are 410 different batches to re-fit models based on the length of our train data. If you do not understand what this means, don't worry!

### Output

Great! That was so easy, wasn't it? Finally, let's see the outputs of our models.

In [7]:
print(ensembled)

        location       date  incidence  incidence_preds
0         1001.0 2020-02-25        0.0              NaN
1         1001.0 2020-02-26        0.0              NaN
2         1001.0 2020-02-27        0.0              NaN
3         1001.0 2020-02-28        0.0              NaN
4         1001.0 2020-02-29        0.0              NaN
...          ...        ...        ...              ...
1541230  56045.0 2021-08-28        NaN        14.313992
1541231  56045.0 2021-08-29        NaN        14.342849
1541232  56045.0 2021-08-30        NaN        14.371654
1541233  56045.0 2021-08-31        NaN        14.399930
1541234  56045.0 2021-09-01        NaN        14.428529

[1541235 rows x 4 columns]


Nicely done! As you can see, although we trained on two models, there is only one output column (incidence_preds) because the `Ensemble` has taken care of them.

Next, let's try to print this model more nicely where the predicted values are intergrated with our incident cases...

In [8]:
ensembled.get_predict(in_sample=True)

,location,date,incidence
0,1001.0,2020-02-25,0.000000
1,1001.0,2020-02-26,0.000000
2,1001.0,2020-02-27,0.000000
3,1001.0,2020-02-28,0.000000
4,1001.0,2020-02-29,0.000000
...,...,...,...
1541230,56045.0,2021-08-28,14.313992
1541231,56045.0,2021-08-29,14.342849
1541232,56045.0,2021-08-30,14.371654
1541233,56045.0,2021-08-31,14.399930


Awesome! The argument `in_sample` means "Do you want to include the ground-truth data in the final output or not?"

You can also save this output into a CSV file.

In [9]:
ensembled.save_predict('first_try.csv', in_sample=True)

## Tips

- ARIMA is a statistical model, it may take much longer time to train and it should require different models for different time-series for best performance (due to its natural inability to "train_on_batch").

- RNNs models such as LSTM, GRU are usually faster to train, convenient enough to have one model for every location, easy to be saved and reloaded for forecasting, refitting with new data as incremental learning, or applying to similar time-series as transfer learning. However, they may be more susceptible to high variance (overfitting). We are also testing an automatic hypertuner to mitigate this.

- In many cases, ARIMA may outperform ML models in forecasting time-series: Mbah, T.J., Ye, H., Zhang, J. et al. Using LSTM and ARIMA to Simulate and Predict Limestone Price Variations. Mining, Metallurgy & Exploration 38, 913–926 (2021). https://doi.org/10.1007/s42461-020-00362-y. Choose your models wisely based on your computational power, use case, and comfort!

- Also do not forget to check our documentations for more advanced applications: train, export, load individual models.

Thank you.